In [1]:
!pip3 install fuzzywuzzy

In [2]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.0 MB/s eta 0:00:00


In [3]:
import requests
from bs4 import BeautifulSoup

paronyms = []


# взятие паронимов с открытого ресурса
home_url = 'https://ru-paronym.ru'
ru_alphabet = [chr(i) for i in range(ord('А'), ord('А') + 32)]
no_words = ['Ё', 'Й', 'Щ', 'Ъ', 'Ы', 'Ь']


for letter in ru_alphabet:
    if letter in no_words:
        continue
    url = home_url + '/' + letter
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    arr = soup.find_all('a', href=True)[2:-27]  # Откидываем первые 2 и последние 27 элемента, которые есть на каждой странице
    for elem in arr:
        webpage = home_url + elem['href']
        words = (elem.string).split(' — ')
        paronyms.append(((words[0], words[1]), 1))  # пары слов - паронимы
        paronyms.append(((words[1], words[0]), 1))  # пара в обратном порядке





In [5]:
# база данных с кортежами с меткой (1) - паронимы
paronyms[:20]

[(('абонемент', 'абонент'), 1),
 (('абонент', 'абонемент'), 1),
 (('абстрактность', 'абстракция'), 1),
 (('абстракция', 'абстрактность'), 1),
 (('авантюрин', 'авантюрист'), 1),
 (('авантюрист', 'авантюрин'), 1),
 (('автобиографический', 'автобиографичный'), 1),
 (('автобиографичный', 'автобиографический'), 1),
 (('автоматический', 'автоматичный'), 1),
 (('автоматичный', 'автоматический'), 1),
 (('автономия', 'автономность'), 1),
 (('автономность', 'автономия'), 1),
 (('авторитарный', 'авторитетный'), 1),
 (('авторитетный', 'авторитарный'), 1),
 (('адресант', 'адресат'), 1),
 (('адресат', 'адресант'), 1),
 (('азарт', 'азартность'), 1),
 (('азартность', 'азарт'), 1),
 (('академический', 'академичный'), 1),
 (('академичный', 'академический'), 1)]

In [ ]:
import random


# создание пар слов не паронимов путем перемешивания уже имеющихся слов
def generate_non_paronyms(paronyms, seed = None):
    all_words = set()

    # Создаем множество всех слов из датасета паронимов
    for pair, _ in paronyms:
        all_words.update(pair)

    non_paronyms = []

    if seed:
      random.seed(seed)

    # Случайные пары из множества всех слов
    for _ in range(len(paronyms)):
        word1, word2 = random.choices(list(all_words), k=2)
        if ((word1, word2), 1) not in paronyms:
          non_paronyms.append(((word1, word2), 0))  # Метка 0, так как это непаронимы

    return non_paronyms

non_paronyms = generate_non_paronyms(paronyms, 42)


In [ ]:
dataset = paronyms + non_paronyms
random.shuffle(dataset) # перемешивание датасета на всякий случай
dataset[:10]

[(('собственнический', 'собственный'), 1),
 (('земной', 'земляной'), 1),
 (('пластинчатый', 'плавки'), 0),
 (('токсичный', 'токсический'), 1),
 (('обязать', 'обвязать'), 1),
 (('публицистичный', 'движитель'), 0),
 (('иструхлявить', 'переродниться'), 0),
 (('кормильщица', 'кормилица'), 1),
 (('взаимопонимание', 'понимание'), 1),
 (('дифференцированный', 'тягостно'), 0)]

In [ ]:
import pandas as pd

data = pd.DataFrame(dataset, columns=['word_pair', 'label'])

In [ ]:
from fuzzywuzzy import fuzz
import Levenshtein

def levenshtein_distance(dataset): # вычисление расстояния левенштейна и сходства слов
    distances = []

    for (word1, word2), _ in dataset:
        lev_dist = Levenshtein.distance(word1, word2)
        lev_ratio = fuzz.ratio(word1, word2)
        distances.append((word1, word2, lev_dist, lev_ratio))

    return distances

In [ ]:
lev = levenshtein_distance(dataset)
lev

[('собственнический', 'собственный', 6, 74),
 ('земной', 'земляной', 2, 86),
 ('пластинчатый', 'плавки', 8, 44),
 ('токсичный', 'токсический', 4, 70),
 ('обязать', 'обвязать', 1, 93),
 ('публицистичный', 'движитель', 11, 26),
 ('иструхлявить', 'переродниться', 10, 32),
 ('кормильщица', 'кормилица', 2, 90),
 ('взаимопонимание', 'понимание', 6, 75),
 ('дифференцированный', 'тягостно', 16, 15),
 ('обезденежить', 'обезденежеть', 1, 92),
 ('исходный', 'исходящий', 3, 71),
 ('метафизичный', 'метафизический', 4, 77),
 ('азартность', 'разместить', 7, 50),
 ('пьянеть', 'школьный', 8, 27),
 ('рогастый', 'мелодический', 9, 30),
 ('остервенеть', 'собирательный', 11, 33),
 ('автоматичный', 'автоматический', 4, 77),
 ('кокетка', 'кокотка', 1, 86),
 ('уклонение', 'отклонение', 2, 84),
 ('оппортунистический', 'нафталин', 15, 23),
 ('протрезвить', 'протрезветь', 1, 91),
 ('прозаический', 'прозаичный', 4, 73),
 ('ограниченность', 'обесплодеть', 11, 40),
 ('проследовать', 'последовать', 1, 96),
 ('фантас